In [1]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import cv2
import tensorflow as tf
from PIL import Image
import os

from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
from keras.models import Sequential, load_model
from keras.layers import Conv2D, MaxPool2D, Dense, Flatten, Dropout

In [2]:
print (os.listdir())
#covid 0, normal 1

['.ipynb_checkpoints', 'Augmentations', 'Breast Cancer ImageDataGen style.ipynb', 'Image Augmentation Code.ipynb', 'model', 'model2', 'modelpush', 'python.py', 'static', 'templates', 'Test', 'Train', 'Ultrasound Classification Model 30,30.ipynb', 'us-dataset.zip']


In [3]:
data = []
labels = []
classes = 3

In [4]:
for i in range(classes):
    path = os.path.join('D:/DATA SCIENCE/DataSets/Research/Breast Cancer Ultrasound','Train',str(i))
    images = os.listdir(path)
    for a in images:
        try:
            image = Image.open(path + '/'+ a).convert('RGB')
            image = image.resize((30,30))
            image = np.array(image)
            data.append(image)
            labels.append(i)
        except Exception as e:
            print(e)

In [5]:
data = np.array(data)
labels = np.array(labels)

In [6]:
data

array([[[[146, 146, 146],
         [185, 185, 185],
         [169, 169, 169],
         ...,
         [188, 188, 188],
         [181, 181, 181],
         [179, 179, 179]],

        [[144, 144, 144],
         [139, 139, 139],
         [113, 113, 113],
         ...,
         [142, 142, 142],
         [155, 155, 155],
         [159, 159, 159]],

        [[134, 134, 134],
         [135, 135, 135],
         [122, 122, 122],
         ...,
         [153, 153, 153],
         [144, 144, 144],
         [117, 117, 117]],

        ...,

        [[117, 117, 117],
         [116, 116, 116],
         [114, 114, 114],
         ...,
         [ 64,  64,  64],
         [ 61,  61,  61],
         [ 62,  62,  62]],

        [[104, 104, 104],
         [104, 104, 104],
         [ 99,  99,  99],
         ...,
         [ 73,  73,  73],
         [ 65,  65,  65],
         [ 63,  63,  63]],

        [[101, 101, 101],
         [ 98,  98,  98],
         [ 90,  90,  90],
         ...,
         [ 42,  42,  42],
        

In [7]:
data = data / data.max()

In [8]:
data

array([[[[0.57254902, 0.57254902, 0.57254902],
         [0.7254902 , 0.7254902 , 0.7254902 ],
         [0.6627451 , 0.6627451 , 0.6627451 ],
         ...,
         [0.7372549 , 0.7372549 , 0.7372549 ],
         [0.70980392, 0.70980392, 0.70980392],
         [0.70196078, 0.70196078, 0.70196078]],

        [[0.56470588, 0.56470588, 0.56470588],
         [0.54509804, 0.54509804, 0.54509804],
         [0.44313725, 0.44313725, 0.44313725],
         ...,
         [0.55686275, 0.55686275, 0.55686275],
         [0.60784314, 0.60784314, 0.60784314],
         [0.62352941, 0.62352941, 0.62352941]],

        [[0.5254902 , 0.5254902 , 0.5254902 ],
         [0.52941176, 0.52941176, 0.52941176],
         [0.47843137, 0.47843137, 0.47843137],
         ...,
         [0.6       , 0.6       , 0.6       ],
         [0.56470588, 0.56470588, 0.56470588],
         [0.45882353, 0.45882353, 0.45882353]],

        ...,

        [[0.45882353, 0.45882353, 0.45882353],
         [0.45490196, 0.45490196, 0.45490196]

In [9]:
labels

array([0, 0, 0, ..., 2, 2, 2])

In [10]:
os.mkdir('modelimprovement')

np.save('./model/data',data)
np.save('./model/target',labels)

In [11]:
data=np.load('./model/data.npy')
labels=np.load('./model/target.npy')

In [12]:
print(data.shape, labels.shape)

(1500, 30, 30, 3) (1500,)


In [13]:
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=45)

In [14]:
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(1200, 30, 30, 3) (300, 30, 30, 3) (1200,) (300,)


## Convert labels to onehot encoding

In [15]:
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

In [16]:
y_test

array([[1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 1., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.]], dtype=float32)

In [18]:
model = Sequential()
model.add(Conv2D(filters=32, kernel_size=(5,5), activation='relu', input_shape=X_train.shape[1:]))
model.add(Conv2D(filters=32, kernel_size=(5,5), activation='relu'))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Dropout(rate=0.35))
model.add(Conv2D(filters=64, kernel_size=(3, 3), activation='relu'))
model.add(Conv2D(filters=64, kernel_size=(3, 3), activation='relu'))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Dropout(rate=0.25))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Dropout(rate=0.15))
model.add(Flatten())
model.add(Dense(30, activation='relu'))
model.add(Dropout(rate=0.10))
model.add(Dense(10, activation='softmax'))

In [19]:
from tensorflow.keras.optimizers import Adam
# Learning Rate and Decay Rate: Reduce the learning rate, a good starting value is usually between 0.0005 to 0.001.
# Also consider a decay rate of 1e-6.

In [20]:
#lr= 0.005
epochs = 50

In [21]:
#opt = Adam(lr=lr, decay=lr/epochs)
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [22]:
history = model.fit(X_train, y_train, batch_size=50, epochs=epochs, validation_data=(X_test, y_test))

Epoch 1/50
24/24 [==============================] - 4s 132ms/step - loss: 2.0303 - accuracy: 0.2646 - val_loss: 1.7835 - val_accuracy: 0.3300
Epoch 2/50
24/24 [==============================] - 3s 122ms/step - loss: 1.5717 - accuracy: 0.3509 - val_loss: 1.4860 - val_accuracy: 0.3433
Epoch 3/50
24/24 [==============================] - 3s 123ms/step - loss: 1.3762 - accuracy: 0.3462 - val_loss: 1.2444 - val_accuracy: 0.3367
Epoch 4/50
24/24 [==============================] - 3s 123ms/step - loss: 1.2904 - accuracy: 0.3412 - val_loss: 1.2512 - val_accuracy: 0.4267
Epoch 5/50
24/24 [==============================] - 3s 123ms/step - loss: 1.2370 - accuracy: 0.3640 - val_loss: 1.1473 - val_accuracy: 0.3800
Epoch 6/50
24/24 [==============================] - 3s 144ms/step - loss: 1.2495 - accuracy: 0.3532 - val_loss: 1.1635 - val_accuracy: 0.5300
Epoch 7/50
24/24 [==============================] - 4s 149ms/step - loss: 1.1472 - accuracy: 0.3964 - val_loss: 1.0796 - val_accuracy: 0.4467
Epoch 

In [ ]:
# accuracy 
plt.figure(0)
plt.plot(history.history['accuracy'], label='training accuracy')
plt.plot(history.history['val_accuracy'], label='val accuracy')
plt.title('Accuracy')
plt.xlabel('epochs')
plt.ylabel('accuracy')
plt.legend()
plt.show()

In [ ]:
# Loss
plt.plot(history.history['loss'], label='training loss')
plt.plot(history.history['val_loss'], label='val loss')
plt.title('Loss')
plt.xlabel('epochs')
plt.ylabel('loss')
plt.legend()
plt.show()

## LEt's do testing on Test data

## Save the model

In [ ]:
model.save("./modelimprovement/model.h5")

## Load the Model

In [ ]:
import os
os.chdir(r'D:\DATA SCIENCE\DataSets\Research\Projects')
from keras.models import load_model
model = load_model(r'D:\DATA SCIENCE\DataSets\Research\Breast Cancer Ultrasound\model\model.h5')

In [ ]:
classes = { 0:'Benign',
            1:'Malignant',
            2:'Normal'}

In [ ]:
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
def test_on_img(img):
    data=[]
    image = Image.open(img).convert('RGB')
    image = image.resize((30,30))
    data.append(np.array(image))
    X_test=np.array(data)
    Y_pred = model.predict_classes(X_test)
    return image,Y_pred

In [ ]:
plot,prediction = test_on_img(r'D:\DATA SCIENCE\DataSets\Research\Breast Cancer Ultrasound\Test\0\benign (95).png')
s = [str(i) for i in prediction] 
a = int("".join(s)) 
print("Predicted Ultra sound image is : ", classes[a])
plt.imshow(plot)
plt.show()

In [ ]:
plot,prediction = test_on_img(r'D:\DATA SCIENCE\DataSets\Research\Breast Cancer Ultrasound\Test\1\malignant (203).png')
s = [str(i) for i in prediction] 
a = int("".join(s)) 
print("Predicted Ultra sound image is : ", classes[a])
plt.imshow(plot)
plt.show()
# 1 wrong prediction

In [ ]:
plot,prediction = test_on_img(r'D:\DATA SCIENCE\DataSets\Research\Breast Cancer Ultrasound\Test\2\normal (133).png')
s = [str(i) for i in prediction] 
a = int("".join(s)) 
print("Predicted Ultra sound image is : ", classes[a])
plt.imshow(plot)
plt.show()

In [ ]:
folder_path=(r'D:\DATA SCIENCE\DataSets\Research\Breast Cancer Ultrasound\Test\2')

In [ ]:
# load all images into a list
images = []
for img in os.listdir(folder_path):
    img = os.path.join(folder_path, img)
    image = Image.open(img)
    image = image.resize((30,30))
    image = np.expand_dims(image, axis=0)
    images.append(np.array(image))

In [ ]:
# stack up images list to pass for prediction
images = np.vstack(images)
classes = model.predict_classes(images, batch_size=10)
print(classes)

In [ ]:
#30,30 model, Above tests are for the models performance as mentioned below
#Epoch 50/50
#33/33 [==============================] - 3s 101ms/step - loss: 0.3501 - accuracy: 0.8667 - val_loss: 0.4979 - val_accuracy: 0.8217